In [9]:
from datasets import load_dataset

dataset = load_dataset("nielsr/funsd")

dataset


Generating train split: 149 examples [00:01, 98.64 examples/s] 
Generating test split: 50 examples [00:00, 116.88 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 50
    })
})

In [10]:
print("Train dataset:: ", len(dataset['train']))
print("Test dataset:: ", len(dataset['test']))

Train dataset::  149
Test dataset::  50
